In [9]:
import keras
from keras.layers.core import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [3]:
train_path='train'
test_path= 'test'

In [15]:
#Vytvoření batches pro train a test

In [4]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['jablka','rajcata'], batch_size=4)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['jablka','rajcata'],batch_size=4)

Found 40 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [16]:
#import VGG16 modelu s vahami, odstranění poslední vrstvy

In [5]:
vgg16_model = VGG16(include_top=False, input_shape=(224, 224, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
#přidání flatten vrstvy po konvoluci, která byla doteď poslední v pořadí, přidání mezivrstvy se 64 neurony
#a poslední vrstvy pro klasifikaci se 2 neurony

In [6]:
flat1 = Flatten()(vgg16_model.outputs)
class1 = Dense(64, activation='relu')(flat1)
output = Dense(2, activation='softmax')(class1)

In [ ]:
#vytvoření nového modelu, který vezme VGG16 model bez poslední vrstvy 
#a nasadí na něj vrstvy definované o buňku výše

In [10]:
clf = Model(inputs=vgg16_model.inputs, outputs=output)

In [ ]:
#zafixování vah původních vrstev z VGG16
#trénovat se budou váhy jen pro poslední tři vrstvy, které byly definovány nově

In [11]:
for layer in clf.layers:
    layer.trainable = False
for layer in clf.layers[-3:]:
    layer.trainable = True

In [12]:
clf.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
#compile modelu

In [13]:
clf.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
#trénování modelu

In [14]:
clf.fit_generator(train_batches, steps_per_epoch=4, validation_data=test_batches, validation_steps=4, epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5


InvalidArgumentError: Matrix size-incompatible: In[0]: [1,100352], In[1]: [25088,64]
	 [[{{node dense_1/MatMul}}]]